In [1]:
(defclass polynomial ()
  ((poly :accessor poly
         :initform nil
         :initarg :poly)
   (deg :accessor poly-deg
        :initform 0
        :initarg :deg)
   )
)

#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>

In [2]:
;;; Make a polynomial taking advantage of program = data
;;; new polynomial term of the form a * x^b:
(defun make-polynomial (a b)
;;     (print (append b (cons * a)))
     (MAKE-INSTANCE 'polynomial :poly  `(+ (* ,a (expt (x ,b)))) :deg b)
)

MAKE-POLYNOMIAL

In [3]:
;;;; TEST of make-polynomial
(make-polynomial 2 3)

#<POLYNOMIAL {1005599623}>

In [4]:
(defmethod degree ((p polynomial))
    (labels ( (degree-1 (poly max)
;;                         (print poly)
;;                         (format t "~C max: ~d calc: ~d " #\linefeed max (caddr (caddar poly)) )
                (cond  ( (null poly) max ) 
                        ( (> (caddr (caddar poly)) max) (degree-1 (cdr poly) (caddr (caddar poly))) )
                        (t (degree-1 (cdr poly) max))
                )
               )
             )
            (degree-1 (cdr (slot-value p 'poly)) 0)
     )
    
    
    )

IMPLICIT-GENERIC-FUNCTION-WARNING: 
  Implicitly creating new generic function CL-JUPYTER-USER::DEGREE.


#<STANDARD-METHOD CL-JUPYTER-USER::DEGREE (POLYNOMIAL) {1005844CD3}>

In [5]:
;;; TESTs of degree
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 7)) (* 3 (expt x 3))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 10)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
      )
     ( print (degree t1))
     ( print (degree t2))
     ( print (degree t3))
     
     )


3 
7 
10 

10

In [8]:
(defmethod print-object ((poly polynomial) out)
    (labels ((to-string (stack term poly  )
                (print "------------------------------------")
                (print stack)
                (print  term)
                (print poly)
                (cond  ((null term) (to-string-1 (reverse stack) "" nil ))
                       ((atom term) term)
                       ((eql (car term) '*) (to-string (push `(,(to-string stack (cadr term) poly) ,(to-string stack (caddr term) poly) ) stack) (car poly) (cdr poly) ) )
                       ((eql (car term) 'expt) `(,(cadr term),(caddr term)))
;;                        (t (to-string-1 stack))
                 )
;;                 (cond ()
;; ;;                       (t )
;;                       (t (print "Finished") stack)
;;                  )
            
;;               (push (pop poly) stack)
;;                         (print stack)
              )
              (to-string-1 (stack res plus)
                           (print "AT to-string-1")
                           (print stack)
                           (format t "res: ~s" res)
                           (print (car stack))
                           (let ((coef (caar stack))
                                 (var  (caadar stack))
                                 (exp  (car (cdadar stack)))
                                 )
                                 (format t "term: ~a~a~a~C" coef var exp #\linefeed)
                                 (when (not (null coef)) (format t "T: ~:[ - ~; + ~]" (> coef 0)))
                                 (cond 
                                       ((null stack) (format t "stack nil "))
                                       ((= coef 0) (format t "0"))
                                       ((= exp 0)  (format t "~a"  (abs coef)))
                                       ((and (= exp 1) (= (abs coef) 1)) (format t "~a" var )  )
                                       ((= (abs coef) 1) (format t "~a^~a "  var exp ) )
                                       ((= exp 1) (format t "~a~a" (abs coef) var))
                                       (t (format t "~a~a~a" (abs coef) var exp ))
;;                                        ((= (caar stack) 1) (format t "T: ~:[+~;~]~a~a" (equal res "") (caadar stack) (car (cdadar stack))) )
;;                                  (t (format t "T: ~@[~a~]~@[~a~]~@[^~a~] + ~@[~a~]~@[~a~]~@[^~a~]" 
;;                                         (when (> (caar stack) 1) (caar stack) )  (when (/= 0 (car  (cdadar stack))) (caadar stack))       (when (> (car  (cdadar stack)) 1) (car (cdadar stack)) )  
;;                                         (when (> (caadr stack) 1) (caadr stack)) (when (/= 0 (cadr (cadadr stack))) (car (cadadr stack))) (when (> (cadr (cadadr stack)) 1) (cadr (cadadr stack)) )
;;                                      )
;;                                   (t (print "finished"))
                                     
;;                                     (to-string-1 
;;                                         (cddr stack) 
;;                                         res)
                            )  (when (not (null stack)) (to-string-1 (cdr stack) res plus) )
                        )
                    )
;;                            (format t "~s^~d" (cadr stack) (car stack) )
                           
;;                            (cond ((eql (car (last stack)) 'expt) (print "eql")  `(,(cadr stack) ^ ,(car stack))) ;(format t "~s^~d" (cadr stack) (car stack) ))
;;                                  ((eql (car (last ))))
;;                                (t (print "OTHER :s"))
;;                               ) )
             
             ) 
            ;;     (format out "~d" (to-string poly) )
            (to-string '() (cadr (slot-value poly 'poly)) (cddr (slot-value poly 'poly) )  ) 
     )
    
    

      
    )

#<STANDARD-METHOD COMMON-LISP:PRINT-OBJECT (POLYNOMIAL T) {10064A6E73}>

In [9]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 0  (expt x 2 ))
                                                 (* 1  (expt x 1 )) 
                                                 (* -1 (expt x 1 )) 
                                                 (* 1  (expt x 2 )) 
                                                 (* -1 (expt x 2 ))
                                                 (* 1  (expt x 0 ))
                                                 (* 2  (expt x 0 )) 
                                                 (* -1 (expt x 0 ))
                                                 (* 2  (expt x 1 )) 
                                                  ) :deg 9)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 0 )) (* 2 (expt x 7)) (* 3 (expt x 3))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 10)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
      )
     ( print t1)
;;      ( print t2)
;;      ( print t3)
     
     )



"------------------------------------" 
NIL 
(* 0 (EXPT X 2)) 
((* 1 (EXPT X 1)) (* -1 (EXPT X 1)) (* 1 (EXPT X 2)) (* -1 (EXPT X 2))
  (* 1 (EXPT X 0)) (* 2 (EXPT X 0)) (* -1 (EXPT X 0)) (* 2 (EXPT X 1))) 
"------------------------------------" 
NIL 
0 
((* 1 (EXPT X 1)) (* -1 (EXPT X 1)) (* 1 (EXPT X 2)) (* -1 (EXPT X 2))
  (* 1 (EXPT X 0)) (* 2 (EXPT X 0)) (* -1 (EXPT X 0)) (* 2 (EXPT X 1))) 
"------------------------------------" 
NIL 
(EXPT X 2) 
((* 1 (EXPT X 1)) (* -1 (EXPT X 1)) (* 1 (EXPT X 2)) (* -1 (EXPT X 2))
  (* 1 (EXPT X 0)) (* 2 (EXPT X 0)) (* -1 (EXPT X 0)) (* 2 (EXPT X 1))) 
"------------------------------------" 
((0 (X 2))) 
(* 1 (EXPT X 1)) 
((* -1 (EXPT X 1)) (* 1 (EXPT X 2)) (* -1 (EXPT X 2)) (* 1 (EXPT X 0))
  (* 2 (EXPT X 0)) (* -1 (EXPT X 0)) (* 2 (EXPT X 1))) 
"------------------------------------" 
((0 (X 2))) 
1 
((* -1 (EXPT X 1)) (* 1 (EXPT X 2)) (* -1 (EXPT X 2)) (* 1 (EXPT X 0))
  (* 2 (EXPT X 0)) (* -1 (EXPT X 0)) (* 2 (EXPT X 1))) 
"----------------

SIMPLE-STYLE-WARNING: 
  The variable T2 is defined but never used.

SIMPLE-STYLE-WARNING: 
  The variable T3 is defined but never used.


In [ ]:
(defvar *zero*)
(defvar *p1*)
(defvar *p2*)
(defvar *p3*)
(defvar *p4*)

(defvar *test*)

(setq *zero* (make-polynomial 0 0))
(setq *p1* (make-polynomial 4 3))
(setq *p2* (make-polynomial 3 2))
(setq *p3* (make-polynomial 1 0))
(setq *p4* (make-polynomial 2 1))

(setq *test* (make-polynomial 5 5))
;; (make-polynomial 4 3)